In [1]:
!pip3 install --upgrade pip --user
!pip3 install kfp --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 1.5 MB 6.9 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.3
    Uninstalling pip-20.3:
      Successfully uninstalled pip-20.3
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [1]:
import kfp
from kfp import compiler
import kfp.components as comp
import kfp.dsl as dsl
from kfp import gcp
import os
from pathlib import Path
import tensorflow as tf

## Define an experiment name

In [2]:
EXPERIMENT_NAME = 'ners'

## Load components from URL

In [133]:
preprocess_operation = kfp.components.load_component_from_url('https://storage.googleapis.com/kfsconfig/componentz/preprocess/component.yaml')
help(preprocess_operation)

training_operation = kfp.components.load_component_from_url('https://storage.googleapis.com/kfsconfig/componentz/training/component.yaml')
help(training_operation)

testing_operation = kfp.components.load_component_from_url('https://storage.googleapis.com/kfsconfig/componentz/testing/component.yaml')
help(testing_operation)

Help on function Convert doccano fomart to spacy:

Convert doccano fomart to spacy(input_data_path:'GCSPath', output_model_uri:'GCSPath')
    Convert doccano fomart to spacy
    Convert doccano fomart to spacy

Help on function Train ner model:

Train ner model(input_data_path:'GCSPath', output_model_uri:'GCSPath', iterations:int)
    Train ner model
    Train ner model on training data

Help on function Test ner model:

Test ner model(input_data_path:'GCSPath', output_model_uri:'GCSPath', model_dir_uri:'GCSPath')
    Test ner model
    Test ner model on test data



## Define the Pipeline

In [134]:
@dsl.pipeline(
  name='new anomaly Pipeline',
  description='Performs preprocessing, training and deployment.')

def pipeline():
    
    preprocess = preprocess_operation(
        input_data_path='ner_data/training_data.json1',
        output_model_uri='data/training_data.txt',)

    training = training_operation(
        input_data_path=preprocess.outputs['output_model_uri'],#'data/training_data.txt',#
        output_model_uri='/model',
        iterations=1)
    
    testing = testing_operation(
        input_data_path='ner_data/test_data.json1',
        output_model_uri='/result',
        model_dir_uri=os.path.join(f"{training.outputs['output_model_uri']}/ner_model"),
                                  )#/model/ner_model

In [135]:
pipeline_func = pipeline
pipeline_filename = pipeline_func.__name__ + '.nerspipeline.zip'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

## Run the Experiment

In [136]:
client = kfp.Client()

try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)
    
print(experiment)

{'created_at': datetime.datetime(2020, 12, 10, 20, 39, 18, tzinfo=tzlocal()),
 'description': None,
 'id': 'cca132fe-e993-40df-ab13-89f5bfd173ff',
 'name': 'ners',
 'resource_references': None,
 'storage_state': None}


## Run pipeline and Experiment

In [137]:
arguments = {}

run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, 
                                 run_name, 
                                 pipeline_filename, 
                                 arguments)

print(experiment.id)
print(run_name)
print(pipeline_filename)
print(arguments)

cca132fe-e993-40df-ab13-89f5bfd173ff
pipeline run
pipeline.nerspipeline.zip
{}
